In [20]:
#### Restructuración del código de clasificacion SIPRE (QUEJAS-DENUNICAS-UNIDAD)
### Ladino Álvarez Ricardo Arturo

### Librerias
#> Base

import re
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from io import StringIO

#> NLP
import spacy
from nltk.corpus import stopwords
nlp = spacy.load('es_core_news_sm')
stopword = set(stopwords.words('spanish'))
from nltk.stem.snowball import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

#> Machine Learning
import xgboost as xgb
from imblearn.over_sampling import SMOTEN
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,mean_squared_error

#> Desbalance de clase
from collections import Counter
from imblearn.over_sampling import SMOTEN


#> Errores
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Entradas = 'C:/Users/LAAR8976/Ladino_ALL/CECTI/CLASIFICADOR_DENUNCIAS_QUEJAS/'

In [ ]:
Quejas = pd.read_excel(Entradas + 'Quejas.xlsx')
Denucias = pd.read_excel(Entradas + 'Denucias.xlsx')
print(Quejas.shape)
print(Denucias.shape)

In [ ]:
Quejas.columns

In [ ]:
Denucias.columns

In [ ]:
# Validar que ambas bases tienen las mismas columnas
if list(Quejas.columns) == list(Denucias.columns):
    print("Ambas bases tienen los mismos nombres de columnas. Puedes concatenarlas.")
    # Concatenar si la validación es exitosa
    datos_concatenados = pd.concat([Quejas, Denucias], ignore_index=True)
    print("Concatenación exitosa.")
else:
    print("Las columnas no coinciden entre las bases. Revisa los nombres de las columnas.")
    print("Columnas en Quejas:", Quejas.columns.tolist())
    print("Columnas en Denuncias:", Denucias.columns.tolist())

In [ ]:
datos_concatenados.shape

### Sección con Pickle

In [ ]:
pickle.dump(datos_concatenados, open("datos_concatenados.pkl", "wb"))

In [29]:
#### importar datos concatenados:

datos_concatenados = pd.read_pickle('datos_concatenados.pkl')

In [30]:
Remuestreo = datos_concatenados[['Consecutivo', 'Folio SIPRE', 'Descripción de los Hechos', 
                      'Tipo asunto', 'Clasificación', 'AG Asignada', 'UA Asignada']]
Remuestreo['Tipo asunto'].value_counts()

Tipo asunto
Denuncia    106203
Queja        40118
Name: count, dtype: int64

### Representatividad de la información

In [31]:
### Función de representatividad

def Repre_clase(Clase_Size):
    if Clase_Size < 500:
        return 1.0  # 100%
    elif 500 <= Clase_Size <= 5000:
        return 0.8  # 80%
    else:
        return 0.5  # 50%
        
Muestras = Remuestreo['Tipo asunto'].value_counts().apply(Repre_clase)

# Aplicar el muestreo por clase
Datos_Muestra = Remuestreo.groupby('Tipo asunto').apply(lambda x: x.sample(frac = Muestras[x.name],
                                                                            random_state = 42)).reset_index(drop = True)

print(Datos_Muestra.shape)
Datos_Muestra['Tipo asunto'].value_counts()

(73161, 7)


Tipo asunto
Denuncia    53102
Queja       20059
Name: count, dtype: int64

### Catálogos de interpretación

In [32]:
Clases = preprocessing.LabelEncoder()
Datos_Muestra['Tipo_asunto'] = Clases.fit_transform(Datos_Muestra['Tipo asunto'])
Datos_Muestra['UA_Asignada'] = Clases.fit_transform(Datos_Muestra['UA Asignada'])
Datos_Muestra.head(2)

,Consecutivo,Folio SIPRE,Descripción de los Hechos,Tipo asunto,Clasificación,AG Asignada,UA Asignada,Tipo_asunto,UA_Asignada
0,643547,72712,Descripción de los hechos: Recibo de nomina no...,Denuncia,Comprobantes Fiscales,AGAFF,AGAFF,0,11
1,671094,100259,Descripción de los hechos: Se realizó un servi...,Denuncia,Comprobantes Fiscales,AGAFF,AGAFF,0,11


In [33]:
Tipo_asunto = pd.DataFrame(set(zip(Datos_Muestra['Tipo asunto'],
                                      Datos_Muestra['Tipo_asunto'])),
                              columns =['Queja_Denuncia', 'Code']).sort_values(by='Code', ascending = True)

UA_Asignada = pd.DataFrame(set(zip(Datos_Muestra['UA Asignada'],
                                      Datos_Muestra['UA_Asignada'])),
                              columns =['UA_Asignada', 'Code']).sort_values(by='Code', ascending = True)

### Limpieza de texto

In [34]:
#### Función de Normalización del texto

### Limpieza de texto
def limpiar_texto(texto):
    # > 00 - StopWords
    stop_words = set(stopwords.words('spanish'))
    # > 01 - Convertir a minúsculas
    texto = texto.lower()
    # > 02 - Eliminar signos de puntuación y números
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\d+', '', texto)
    
    # > 03 - Eliminar acentos y caracteres especiales
    #texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    
    # > 04 Eliminar stopwords
    texto = ' '.join([palabra for palabra in texto.split() if palabra not in stop_words])
    
    # > 05 Eliminar palabras cortas
    texto = ' '.join([palabra for palabra in texto.split() if len(palabra) > 2])

    # > 06 Corrección ortográfica
    #texto = ' '.join([spell.correction(palabra) for palabra in texto.split()])

    # > 07 Lematización
    #texto = ' '.join([token.lemma_ for token in nlp(texto)])

    return (texto)

### Limpieza de múltiples columnas
def limpiar_columnas(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].astype(str).fillna('')  # Convertir a string y manejar NaNs
        df[columna] = df[columna].apply(limpiar_texto)    # Aplicar la función de limpieza completa
    return (df)

In [35]:
Datos_Muestra_N = Datos_Muestra.copy()
Columnas_Limpia = ['Descripción de los Hechos'] 
Datos_Muestra_N = limpiar_columnas(Datos_Muestra_N, Columnas_Limpia)

### Prueba y entrenamiento

In [44]:
X_train, X_test, y_train, y_test = train_test_split(Datos_Muestra_N['Descripción de los Hechos'],
                                                    Datos_Muestra_N['Tipo_asunto'],
                                                    stratify = Datos_Muestra_N['Tipo_asunto'],
                                                    test_size = 0.60,
                                                    random_state = 42)

### Vector de aprendizaje + Bolsa de palabras

In [45]:
TFIDF = TfidfVectorizer(strip_accents = 'unicode', 
                        analyzer = 'word',
                        max_features = 10000,
                        norm='l2',
                        encoding = 'latin-1',
                        token_pattern = r'\w{1,}',
                        ngram_range = (1, 2), 
                        use_idf = True,
                        smooth_idf = True, 
                        sublinear_tf = True)

In [46]:
TDFT = TFIDF.fit(list(X_train) + list(X_test))
X_train_N =  TDFT.transform(X_train) 
X_test_N = TFIDF.transform(X_test)

In [39]:
pickle.dump(TDFT, open("Palabras_Tipo_Asunto.pkl", "wb"))

In [48]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([21241,  8023]))

In [41]:
pickle.dump(X_train_N, open("X_train_N.pkl", "wb"))
pickle.dump(y_train, open("y_train.pkl", "wb"))
pickle.dump(X_test_N, open("X_test_N.pkl", "wb"))
pickle.dump(y_test, open("y_test.pkl", "wb"))

In [49]:
### ================================= ================================= =============================== ###
### ========================================= QUEJAS & DENUNCIAS ====================================== ###
### ============================== FUNCIÓN DE APRENDIZAJE MAQUINA ===================================== ###

def Aprendizaje_Queja_Denuncia(X_train, X_test, y_train, y_test, Model):
    ''' Función de modelo de aprendizaje, que ejecuta lo siguiente:
        A) Se evaluan diferentes modelos : Random Forest, Naibe Bayes y XGBoost '''
    
    if Model == "Random_Forest":
        
        Ranf_clf = {'n_estimators': [64, 128, 256, 512, 1024],
                    'max_features': ['auto', 'sqrt', 'log2'],
                    'max_depth': [2, 4, 8, 16, 32, 64, 128],
                    'criterion': ['gini', 'entropy']
                   }
        
        estimador_RF = RandomForestClassifier(random_state = 42,
                                              n_jobs = multiprocessing.cpu_count()-1)
        
        RF_CECTI = GridSearchCV(estimator = estimador_RF,
                                param_grid = Ranf_clf,
                                cv = RepeatedKFold(n_splits = 3, 
                                                   n_repeats = 1, 
                                                   random_state = 42),
                                scoring = 'accuracy',
                                refit = True,
                                verbose = 0,
                                return_train_score = True)
        
        CECTI_RF = RF_CECTI.fit(X_train,y_train)
        
        Best_estimador = CECTI_RF.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_RF_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        RF_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("Random Forest > Accuracy: ", np.round((RF_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
        
    elif Model == "Naibe_Bayes":
        
        n_classes = np.unique(y_train)
        
        params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                  'fit_prior': [True, False],
                  'class_prior': [None, [0.1,]* len(n_classes)]
                 }
        
        Esti_MNB = MultinomialNB()
        
        MNB_grid = GridSearchCV(Esti_MNB,
                                param_grid = params, 
                                n_jobs = multiprocessing.cpu_count()-1,
                                cv = RepeatedKFold(n_splits = 3, n_repeats = 1,
                                                   random_state = 42),
                                verbose =0 )
        
        CECTI_MNB = MNB_grid.fit(X_train,y_train)
        
        Best_estimador = CECTI_MNB.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_NB_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        NB_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("Naibe Bayes > Accuracy: ", np.round((NB_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
    
    elif Model == "XGBoost":
        
        xgb_params = {'max_depth':[None, 1, 3, 5, 10, 20, 30],
                      'learning_rate':[0.001, 0.01, 0.1],
                      'n_estimators': [50, 100, 200]
                     }
        
        Esti_XGB = xgb.XGBClassifier(objective = 'multi:softprob')
        
        XGB_grid = GridSearchCV(estimator = Esti_XGB,
                                param_grid = xgb_params,
                                n_jobs = multiprocessing.cpu_count()-1,
                                cv = RepeatedKFold(n_splits = 5, 
                                                   n_repeats = 3, 
                                                   random_state = 42),
                                scoring='accuracy',
                                verbose = 0)
        
        CECTI_XGB = XGB_grid.fit(X_train,y_train)
        
        Best_estimador = CECTI_XGB.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_XGB_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        XG_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("XGBoost > Accuracy: ", np.round((XG_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
    
    return(Best_estimador)


In [50]:
Aprendizaje_Queja_Denuncia(X_train_N, X_test_N, y_train, y_test, "Random_Forest")

----------------------------
----------------------------
Random Forest > Accuracy:  97.638 %
----------------------------
Reporte de clasificación: 
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     31861
           1       0.93      0.99      0.96     12036

    accuracy                           0.98     43897
   macro avg       0.96      0.98      0.97     43897
weighted avg       0.98      0.98      0.98     43897



RandomForestClassifier(max_depth=128, max_features='auto', n_estimators=64,
                       n_jobs=7, random_state=42)

In [58]:
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
import multiprocessing

def Aprendizaje_Queja_Denuncia(X_train, X_test, y_train, y_test):
    """
    Función de modelo de aprendizaje automático que evalúa:
    Random Forest, Naive Bayes y XGBoost,
    seleccionando automáticamente el mejor modelo.
    """
    # Definir los modelos y sus configuraciones
    modelos = {
        "Random_Forest": {
            "estimator": RandomForestClassifier(random_state=42, n_jobs=multiprocessing.cpu_count() - 1),
            "params": {
                'n_estimators': [64, 128, 256, 512, 1024],
                'max_features': ['sqrt', 'log2'],
                'max_depth': [2, 4, 8, 16, 32, 64, 128],
                'criterion': ['gini', 'entropy']
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "Naive_Bayes": {
            "estimator": MultinomialNB(),
            "params": {
                'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                'fit_prior': [True, False],
                'class_prior': [None, [0.1] * len(np.unique(y_train))]
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "XGBoost": {
            "estimator": XGBClassifier(objective='multi:softprob'),
            "params": {
                'max_depth': [None, 1, 3, 5, 10, 20, 30],
                'learning_rate': [0.001, 0.01, 0.1],
                'n_estimators': [50, 100, 200]
            },
            "cv": RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
        }
    }

    # Variables para almacenar el mejor modelo
    mejor_modelo = None
    mejor_accuracy = 0
    mejor_nombre = ""
    
    # Iterar sobre los modelos
    for nombre, config in modelos.items():
        print(f"Entrenando modelo: {nombre}")
        
        # Configurar GridSearchCV
        grid = GridSearchCV(
            estimator=config["estimator"],
            param_grid=config["params"],
            cv=config["cv"],
            scoring='accuracy',
            n_jobs=multiprocessing.cpu_count() - 1,
            verbose=0
        )
        
        # Entrenar el modelo
        grid.fit(X_train, y_train)
        mejor_estimador = grid.best_estimator_
        
        # Evaluar el modelo
        y_pred = mejor_estimador.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        
        print(f"{nombre} > Accuracy: {np.round(accuracy * 100, 3)}%")
        print('Reporte de clasificación:\n', classification_report(y_test, y_pred))
        
        # Actualizar el mejor modelo si tiene mejor rendimiento
        if accuracy > mejor_accuracy:
            mejor_accuracy = accuracy
            mejor_modelo = mejor_estimador
            mejor_nombre = nombre
    
    # Guardar el mejor modelo
    print(f"\nEl mejor modelo es: {mejor_nombre} con una precisión de {np.round(mejor_accuracy * 100, 3)}%")
    pickle.dump(mejor_modelo, open(f"Mejor_Modelo_{mejor_nombre}.pkl", "wb"))
    
    return mejor_modelo


In [60]:
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import multiprocessing

def Aprendizaje_Queja_Denuncia(X_train, X_test, y_train, y_test):
    """
    Función de modelo de aprendizaje automático que evalúa:
    Random Forest, Naive Bayes y XGBoost,
    seleccionando automáticamente el mejor modelo.
    """
    # Verificar que las etiquetas están correctamente codificadas
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)
    
    # Determinar el número de clases
    n_classes = len(np.unique(y_train))
    print(f"Número de clases: {n_classes}")
    
    # Definir los modelos y sus configuraciones
    modelos = {
        "Random_Forest": {
            "estimator": RandomForestClassifier(random_state=42, n_jobs=multiprocessing.cpu_count() - 1),
            "params": {
                'n_estimators': [64, 128, 256, 512, 1024],
                'max_features': ['sqrt', 'log2'],  # 'auto' reemplazado por 'sqrt'
                'max_depth': [2, 4, 8, 16, 32, 64, 128],
                'criterion': ['gini', 'entropy']
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "Naive_Bayes": {
            "estimator": MultinomialNB(),
            "params": {
                'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                'fit_prior': [True, False],
                'class_prior': [None, [0.1] * n_classes]
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "XGBoost": {
            "estimator": XGBClassifier(objective='multi:softprob', num_class=n_classes),
            "params": {
                'max_depth': [1, 3, 5, 10, 20],
                'learning_rate': [0.001, 0.01, 0.1],
                'n_estimators': [50, 100, 200]
            },
            "cv": RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
        }
    }

    # Variables para almacenar el mejor modelo
    mejor_modelo = None
    mejor_accuracy = 0
    mejor_nombre = ""
    
    # Iterar sobre los modelos
    for nombre, config in modelos.items():
        print(f"Entrenando modelo: {nombre}")
        
        # Configurar GridSearchCV con error_score='raise'
        grid = GridSearchCV(
            estimator=config["estimator"],
            param_grid=config["params"],
            cv=config["cv"],
            scoring='accuracy',
            n_jobs=multiprocessing.cpu_count() - 1,
            verbose=0,
            error_score='raise'
        )
        
        # Entrenar el modelo
        grid.fit(X_train, y_train)
        mejor_estimador = grid.best_estimator_
        
        # Evaluar el modelo
        y_pred = mejor_estimador.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_pred)
        
        print(f"{nombre} > Accuracy: {np.round(accuracy * 100, 3)}%")
        print('Reporte de clasificación:\n', classification_report(y_test, y_pred))
        
        # Actualizar el mejor modelo si tiene mejor rendimiento
        if accuracy > mejor_accuracy:
            mejor_accuracy = accuracy
            mejor_modelo = mejor_estimador
            mejor_nombre = nombre
    
    # Guardar el mejor modelo
    print(f"\nEl mejor modelo es: {mejor_nombre} con una precisión de {np.round(mejor_accuracy * 100, 3)}%")
    pickle.dump(mejor_modelo, open(f"Mejor_Modelo_{mejor_nombre}.pkl", "wb"))
    
    return mejor_modelo


In [63]:
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import multiprocessing

def Aprendizaje_Queja_Denuncia(X_train, X_test, y_train, y_test):
    """
    Función de modelo de aprendizaje automático que evalúa:
    Random Forest, Naive Bayes y XGBoost,
    seleccionando automáticamente el mejor modelo.
    """

    # Verificar y ajustar el formato de las etiquetas
    if len(y_train.shape) > 1:  # Etiquetas multilabel-indicator
        print("Las etiquetas están en formato multilabel. Convirtiendo a unidimensional.")
        y_train = np.argmax(y_train, axis=1)
        y_test = np.argmax(y_test, axis=1)
    else:
        print("Etiquetas unidimensionales detectadas. Aplicando codificación si es necesario.")
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
    
    # Determinar el número de clases
    n_classes = len(np.unique(y_train))
    print(f"Número de clases: {n_classes}")
    
    # Definir los modelos y sus configuraciones
    modelos = {
        "Random_Forest": {
            "estimator": RandomForestClassifier(random_state=42, n_jobs=multiprocessing.cpu_count() - 1),
            "params": {
                'n_estimators': [64, 128, 256, 512, 1024],
                'max_features': ['sqrt', 'log2'],  # 'auto' reemplazado por 'sqrt'
                'max_depth': [2, 4, 8, 16, 32, 64, 128],
                'criterion': ['gini', 'entropy']
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "Naive_Bayes": {
            "estimator": MultinomialNB(),
            "params": {
                'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                'fit_prior': [True, False],
                'class_prior': [None, [0.1] * n_classes]
            },
            "cv": RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)
        },
        "XGBoost": {
            "estimator": XGBClassifier(objective='multi:softprob' if n_classes > 2 else 'binary:logistic', 
                                       num_class=n_classes if n_classes > 2 else None),
            "params": {
                'max_depth': [1, 3, 5, 10, 20],
                'learning_rate': [0.001, 0.01, 0.1],
                'n_estimators': [50, 100, 200]
            },
            "cv": RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
        }
    }

    # Variables para almacenar el mejor modelo
    mejor_modelo = None
    mejor_accuracy = 0
    mejor_nombre = ""
    
    # Iterar sobre los modelos
    for nombre, config in modelos.items():
        print(f"\nEntrenando modelo: {nombre}")
        
        # Configurar GridSearchCV con error_score='raise'
        grid = GridSearchCV(
            estimator=config["estimator"],
            param_grid=config["params"],
            cv=config["cv"],
            scoring='accuracy',
            n_jobs=multiprocessing.cpu_count() - 1,
            verbose=0,
            error_score='raise'
        )
        
        try:
            # Entrenar el modelo
            grid.fit(X_train, y_train)
            mejor_estimador = grid.best_estimator_
            
            # Evaluar el modelo
            y_pred = mejor_estimador.predict(X_test)
            accuracy = metrics.accuracy_score(y_test, y_pred)
            
            print(f"{nombre} > Accuracy: {np.round(accuracy * 100, 3)}%")
            print('Reporte de clasificación:\n', classification_report(y_test, y_pred))
            
            # Actualizar el mejor modelo si tiene mejor rendimiento
            if accuracy > mejor_accuracy:
                mejor_accuracy = accuracy
                mejor_modelo = mejor_estimador
                mejor_nombre = nombre
        except Exception as e:
            print(f"Error entrenando el modelo {nombre}: {e}")
    
    # Guardar el mejor modelo
    if mejor_modelo is not None:
        print(f"\nEl mejor modelo es: {mejor_nombre} con una precisión de {np.round(mejor_accuracy * 100, 3)}%")
        pickle.dump(mejor_modelo, open(f"Mejor_Modelo_{mejor_nombre}.pkl", "wb"))
    else:
        print("No se encontró un modelo válido.")
    
    return mejor_modelo


In [ ]:
mejor_modelo = Aprendizaje_Queja_Denuncia(X_train_N, X_test_N, y_train, y_test)

Etiquetas unidimensionales detectadas. Aplicando codificación si es necesario.
Número de clases: 2

Entrenando modelo: Random_Forest
Random_Forest > Accuracy: 97.638%
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     31861
           1       0.93      0.99      0.96     12036

    accuracy                           0.98     43897
   macro avg       0.96      0.98      0.97     43897
weighted avg       0.98      0.98      0.98     43897


Entrenando modelo: Naive_Bayes
Naive_Bayes > Accuracy: 96.07%
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97     31861
           1       0.91      0.95      0.93     12036

    accuracy                           0.96     43897
   macro avg       0.95      0.96      0.95     43897
weighted avg       0.96      0.96      0.96     43897


Entrenando modelo: XGBoost
